In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
from sklearn import preprocessing
from scipy import interpolate
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
# First, import the data:
pd.options.display.max_columns = 60
# pd.options.display.max_rows = None
df_1 = pd.read_csv('fahrzeiten_soll_ist_20160103_20160109.csv')

####################
df_haltepunkt = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltepunkt.csv")
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
# df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
# df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


# Bus Station
df_haltestelle = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltestelle.csv")
####################

def find_dist(lat1, lon1, lat_2, lon2):
    c1 = (lat1, lon1)
    c2 = (lat_2, lon2)
    return geopy.distance.vincenty(c1, c2).km
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################


# Removing Nans


In [2]:
# Drop punkts with no GPS lan and lon

In [3]:
print(df_haltepunkt.info())
df_haltepunkt.dropna(axis=0, how='any', inplace=True, subset=['GPS_Longitude'])
print(df_haltepunkt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322 entries, 0 to 5321
Data columns (total 7 columns):
halt_punkt_id           5322 non-null int64
halt_punkt_diva         5322 non-null int64
halt_id                 5322 non-null int64
GPS_Latitude            4315 non-null object
GPS_Longitude           4315 non-null object
GPS_Bearing             3321 non-null float64
halt_punkt_ist_aktiv    5322 non-null bool
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 254.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4315 entries, 0 to 5321
Data columns (total 7 columns):
halt_punkt_id           4315 non-null int64
halt_punkt_diva         4315 non-null int64
halt_id                 4315 non-null int64
GPS_Latitude            4315 non-null object
GPS_Longitude           4315 non-null object
GPS_Bearing             3317 non-null float64
halt_punkt_ist_aktiv    4315 non-null bool
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 240.2+ KB
None


In [4]:
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


In [5]:
# How many total punkts?

In [6]:
len(set(df_haltepunkt.GPS_Latitude))

3319

In [7]:
# How many punkts in this dataset?

In [7]:
len(set(df_1.halt_punkt_id_von))

1492

In [8]:
df_1 = df_1[df_1['betriebsdatum'] == '05.01.16']
len(df_1)

214040

    Merge the two datasets on haltpunkts - from and to

In [9]:
df_1['halt_punkt_id'] = df_1['halt_punkt_id_von']
df_1 = df_1.merge(df_haltepunkt, on=['halt_punkt_id'])
df_1

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva,halt_id,GPS_Latitude,GPS_Longitude,GPS_Bearing,halt_punkt_ist_aktiv
0,80,2,05.01.16,10421,12,7,2034,22,RAUT,06.01.16,3612,3601,3624,3601,8,1570,22,LIND,06.01.16,3702,3682,3714,3699,97583,12808,24,1,24,TRIE - BALT,113576,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
1,80,2,05.01.16,10409,6,8,2034,22,RAUT,05.01.16,40986,41005,41010,41030,9,1570,22,LIND,05.01.16,41100,41115,41124,41136,96286,23892,2,1,2,TRIS - BOEN,112958,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
2,80,2,05.01.16,10417,1,8,2034,22,RAUT,05.01.16,73776,73811,73788,73821,9,1570,22,LIND,05.01.16,73866,73893,73878,73907,96577,23892,2,1,2,TRIS - BOEN,113230,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
3,80,2,05.01.16,10413,11,7,2034,22,RAUT,05.01.16,22692,22690,22704,22704,8,1570,22,LIND,05.01.16,22782,22773,22794,22782,97528,4827,4,1,4,TRIE - BOEN,112668,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
4,78,1,05.01.16,10263,4,5,2034,22,RAUT,05.01.16,30006,30034,30030,30053,6,1570,22,LIND,05.01.16,30120,30142,30138,30161,54018,4268,1,1,1,DUNK - BAEN,111413,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
5,80,2,05.01.16,10276,19,7,2034,22,RAUT,05.01.16,82392,82452,82404,82460,8,1570,22,LIND,05.01.16,82482,82544,82494,82544,97619,4827,4,1,4,TRIE - BOEN,111701,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
6,80,2,05.01.16,10407,5,8,2034,22,RAUT,05.01.16,60252,60329,60276,60345,9,1570,22,LIND,05.01.16,60366,60430,60390,60454,96272,23892,2,1,2,TRIS - BOEN,113430,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
7,80,2,05.01.16,10427,15,8,2034,22,RAUT,05.01.16,25176,25168,25200,25186,9,1570,22,LIND,05.01.16,25290,25278,25308,25291,97600,23892,2,1,2,TRIS - BOEN,112233,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
8,80,2,05.01.16,10404,8,8,2034,22,RAUT,05.01.16,66684,66646,66708,66662,9,1570,22,LIND,05.01.16,66798,66744,66822,66764,97468,23892,2,1,2,TRIS - BOEN,112211,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False
9,80,2,05.01.16,10416,5,8,2034,22,RAUT,05.01.16,40536,40571,40560,40583,9,1570,22,LIND,05.01.16,40650,40669,40674,40689,96264,23892,2,1,2,TRIS - BOEN,113430,2543,2535,5899,5869,5899,22,2543,47.384008,8.484421,14.0,False


In [10]:
df_1['halt_punkt_id'] = df_1['halt_punkt_id_nach']
df_1 = df_1.merge(df_haltepunkt, on='halt_punkt_id')
df_1

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y
0,80,2,05.01.16,10421,12,7,2034,22,RAUT,06.01.16,3612,3601,3624,3601,8,1570,22,LIND,06.01.16,3702,3682,3714,3699,97583,12808,24,1,24,TRIE - BALT,113576,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
1,80,2,05.01.16,10409,6,8,2034,22,RAUT,05.01.16,40986,41005,41010,41030,9,1570,22,LIND,05.01.16,41100,41115,41124,41136,96286,23892,2,1,2,TRIS - BOEN,112958,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
2,80,2,05.01.16,10417,1,8,2034,22,RAUT,05.01.16,73776,73811,73788,73821,9,1570,22,LIND,05.01.16,73866,73893,73878,73907,96577,23892,2,1,2,TRIS - BOEN,113230,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
3,80,2,05.01.16,10413,11,7,2034,22,RAUT,05.01.16,22692,22690,22704,22704,8,1570,22,LIND,05.01.16,22782,22773,22794,22782,97528,4827,4,1,4,TRIE - BOEN,112668,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
4,78,1,05.01.16,10263,4,5,2034,22,RAUT,05.01.16,30006,30034,30030,30053,6,1570,22,LIND,05.01.16,30120,30142,30138,30161,54018,4268,1,1,1,DUNK - BAEN,111413,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
5,80,2,05.01.16,10276,19,7,2034,22,RAUT,05.01.16,82392,82452,82404,82460,8,1570,22,LIND,05.01.16,82482,82544,82494,82544,97619,4827,4,1,4,TRIE - BOEN,111701,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
6,80,2,05.01.16,10407,5,8,2034,22,RAUT,05.01.16,60252,60329,60276,60345,9,1570,22,LIND,05.01.16,60366,60430,60390,60454,96272,23892,2,1,2,TRIS - BOEN,113430,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
7,80,2,05.01.16,10427,15,8,2034,22,RAUT,05.01.16,25176,25168,25200,25186,9,1570,22,LIND,05.01.16,25290,25278,25308,25291,97600,23892,2,1,2,TRIS - BOEN,112233,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
8,80,2,05.01.16,10404,8,8,2034,22,RAUT,05.01.16,66684,66646,66708,66662,9,1570,22,LIND,05.01.16,66798,66744,66822,66764,97468,23892,2,1,2,TRIS - BOEN,112211,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True
9,80,2,05.01.16,10416,5,8,2034,22,RAUT,05.01.16,40536,40571,40560,40583,9,1570,22,LIND,05.01.16,40650,40669,40674,40689,96264,23892,2,1,2,TRIS - BOEN,113430,2543,2535,5899,5869,5869,22,2543,47.384008,8.484421,14.0,False,22,2535,47.388735,8.486845,356.0,True


In [11]:
b = df_1.sort_values(by = ['betriebsdatum', 'fahrt_id', 'seq_von'])
b = b[b['betriebsdatum'] == b['datum_nach']]
b

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y
102180,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,56190,56215,56190,56215,2,2907,1,WILD,05.01.16,56250,56264,56262,56279,44245,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True
18643,2,2,05.01.16,3016,1,2,2907,1,WILD,05.01.16,56250,56264,56262,56279,3,917,1,FROE,05.01.16,56304,56321,56322,56335,44245,18070,2,1,2,BTIE - FARB,112368,2651,2680,6055,6371,6371,1,2651,47.353376,8.558699,330.0,True,1,2680,47.355373,8.556809,327.0,False
16985,2,2,05.01.16,3016,1,3,917,1,FROE,05.01.16,56304,56321,56322,56335,4,1286,1,HOES,05.01.16,56376,56392,56394,56410,44245,18070,2,1,2,BTIE - FARB,112368,2680,2615,6371,5961,5961,1,2680,47.355373,8.556809,327.0,False,1,2615,47.358061,8.554178,331.0,False
91151,2,2,05.01.16,3016,1,4,1286,1,HOES,05.01.16,56376,56392,56394,56410,5,832,1,FELD,05.01.16,56436,56450,56454,56471,44245,18070,2,1,2,BTIE - FARB,112368,2615,2689,5961,6392,6392,1,2615,47.358061,8.554178,331.0,False,1,2689,47.360292,8.552318,327.0,False
62937,2,2,05.01.16,3016,1,5,832,1,FELD,05.01.16,56436,56450,56454,56471,6,1456,1,KRES,05.01.16,56508,56523,56526,56538,44245,18070,2,1,2,BTIE - FARB,112368,2689,2612,6392,5951,5951,1,2689,47.360292,8.552318,327.0,False,1,2612,47.362914,8.549911,337.0,False
121441,2,2,05.01.16,3016,1,6,1456,1,KRES,05.01.16,56508,56523,56526,56538,7,1845,1,OPER,05.01.16,56598,56609,56616,56632,44245,18070,2,1,2,BTIE - FARB,112368,2612,2104,5951,4814,4814,1,2612,47.362914,8.549911,337.0,False,1,2104,47.365601,8.547372,315.0,False
101700,2,2,05.01.16,3016,1,7,1845,1,OPER,05.01.16,56598,56609,56616,56632,8,440,1,BELL,05.01.16,56706,56704,56736,56726,44245,18070,2,1,2,BTIE - FARB,112368,2104,1565,4814,3552,3552,1,2104,47.365601,8.547372,315.0,False,1,1565,47.366966,8.544832,280.0,False
112250,2,2,05.01.16,3016,1,8,440,1,BELL,05.01.16,56706,56704,56736,56726,9,615,1,BURK,05.01.16,56844,56797,56868,56818,44245,18070,2,1,2,BTIE - FARB,112368,1565,2590,3552,6339,6339,1,1565,47.366966,8.544832,280.0,False,1,2590,47.366463,8.540955,257.0,False
110550,2,2,05.01.16,3016,1,9,615,1,BURK,05.01.16,56844,56797,56868,56818,10,601,1,BOES,05.01.16,56940,56897,56958,56917,44245,18070,2,1,2,BTIE - FARB,112368,2590,2228,6339,6210,6210,1,2590,47.366463,8.540955,257.0,False,1,2228,47.368372,8.539843,346.0,False
108364,2,2,05.01.16,3016,1,10,601,1,BOES,05.01.16,56940,56897,56958,56917,11,1861,1,PARA,05.01.16,57012,56973,57036,57005,44245,18070,2,1,2,BTIE - FARB,112368,2228,1528,6210,3387,3387,1,2228,47.368372,8.539843,346.0,False,1,1528,47.369552,8.538715,243.0,False


In [12]:
# Merge the dataset on haltestelle

In [13]:
b['halt_id'] = b['halt_id_x']
b = b.merge(df_haltestelle, on='halt_id')

In [14]:
b['halt_id'] = b['halt_id_y']
b = b.merge(df_haltestelle, on='halt_id')
b

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y
0,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,56190,56215,56190,56215,2,2907,1,WILD,05.01.16,56250,56264,56262,56279,44245,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
1,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,61080,61096,61080,61096,2,2907,1,WILD,05.01.16,61140,61159,61158,61178,44247,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
2,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,66030,66046,66030,66046,2,2907,1,WILD,05.01.16,66090,66097,66108,66114,44249,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
3,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,71040,71041,71040,71041,2,2907,1,WILD,05.01.16,71100,71111,71112,71128,44251,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
4,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,75300,75303,75300,75303,2,2907,1,WILD,05.01.16,75360,75363,75372,75378,44253,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
5,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,79500,79524,79500,79524,2,2907,1,WILD,05.01.16,79560,79575,79572,79578,44255,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
6,2,2,05.01.16,2069,2,1,3041,0,BTIE,05.01.16,20820,20828,20820,20828,2,2907,1,WILD,05.01.16,20880,20895,20892,20904,44261,18070,2,1,2,BTIE - FARB,111683,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
7,2,2,05.01.16,2069,2,1,3041,0,BTIE,05.01.16,25140,25160,25140,25160,2,2907,1,WILD,05.01.16,25200,25217,25218,25233,44263,18070,2,1,2,BTIE - FARB,111683,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
8,2,2,05.01.16,2069,2,1,3041,0,BTIE,05.01.16,29640,29644,29640,29644,2,2907,1,WILD,05.01.16,29700,29707,29718,29721,44265,18070,2,1,2,BTIE - FARB,111683,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,2651,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",True,2907,WILD,"Zürich, Wildbachstrasse",True
9,2,2,05.01.16,2069,2,1,3041,0,BTIE,05.01.16,34140,34164,34140,34164,2,2907,1,WILD,05.01.16,34200,34221,34212,34230,44267,18070,2,1,2,BTIE - FARB,111683,2788,2651,6

In [15]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209296 entries, 0 to 209295
Data columns (total 56 columns):
linie                     209296 non-null int64
richtung                  209296 non-null int64
betriebsdatum             209296 non-null object
fahrzeug                  209296 non-null int64
kurs                      209296 non-null int64
seq_von                   209296 non-null int64
halt_diva_von             209296 non-null int64
halt_punkt_diva_von       209296 non-null int64
halt_kurz_von1            209296 non-null object
datum_von                 209296 non-null object
soll_an_von               209296 non-null int64
ist_an_von                209296 non-null int64
soll_ab_von               209296 non-null int64
ist_ab_von                209296 non-null int64
seq_nach                  209296 non-null int64
halt_diva_nach            209296 non-null int64
halt_punkt_diva_nach      209296 non-null int64
halt_kurz_nach1           209296 non-null object
datum_nach           

In [16]:
print(len(set(b['halt_id_nach'])), ' many unique haltestelles in the dataset')
print(len(set(b['halt_punkt_id_nach'])), 'many unique haltpunkts in the dataset')

659  many unique haltestelles in the dataset
1489 many unique haltpunkts in the dataset


In [17]:
# Sorting by by the date and departure time
b = df_1.sort_values(by = ['betriebsdatum', 'soll_ab_von', 'fahrt_id', 'seq_von'])
b = b[b['betriebsdatum'] == b['datum_nach']]

In [18]:
# Limiting the intervals to 6 - 12
b = b[(b[target_arrival_from] > 18000) & (b[target_arrival_from] < 86400)]

In [19]:
time_period = 600
starting_time = 18000
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
b['intervals'] = pd.cut(b[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)

In [20]:
# Some Signals
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'
delay = 'delay'

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
b[target_travel_time] = b[target_arrival_to] - b[target_departure_from]
b[delay] = b[actual_arrival_from] - b[target_arrival_from]

b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
# b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

In [21]:
# Top 5 rides with the most number of stops
b['fahrt_id'].value_counts()[:5]

48225    34
47558    34
48233    34
48163    34
48243    34
Name: fahrt_id, dtype: int64

In [22]:
# The most amount of datapoints
temp = b.copy(deep=True)

# pd.options.display.max_columns = 6
pd.options.display.max_rows = 6

# temp = b[b['fahrt_id'] == 47674]
temp

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y,intervals,target_stationary,target_travel_time,delay
192238,17,2,05.01.16,2083,2,4,467,1,BERO,05.01.16,18006,17991,18018,17996,5,2447,1,SPOH,05.01.16,18078,18055,18090,18059,48512,6736,6,1,6,EWYS - WERD für Ausfahrt,114055,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,0,12,60,-15
115965,31,1,05.01.16,10080,5,1,1162,10,HERD,05.01.16,18018,18053,18018,18053,2,1122,10,HARP,05.01.16,18078,18096,18096,18104,49704,21416,9,1,9,HERD - HEGI für Ausfahrt,93414,2868,2526,6842,5792,5792,10,2868,47.384835,8.508979,114.0,False,10,2526,47.382970,8.515127,114.0,False,0,0,60,35
42420,46,2,05.01.16,10147,10,5,2104,11,ROSE,05.01.16,18012,18027,18024,18036,6,2104,13,ROSE,05.01.16,18054,18082,18072,18086,51137,18551,14,1,14,HARP - RUET für Ausfahrt,112441,2749,2749,6543,6544,6544,11,2749,47.394494,8.525626,27.0,False,13,2749,47.394271,8.525343,295.0,False,0,12,30,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107556,701,2,05.01.16,11021,17,11,899,51,FWIT,05.01.16,86340,86331,86352,86331,12,3444,51,GLOK,05.01.16,86376,86361,86388,86361,67879,22268,4,1,4,MSEE - KLUS via MLOR,111106,2466,2426,5656,5559,5559,51,2466,47.361417,8.602384,255.0,True,51,2426,47.360907,8.598600,241.0,True,NaN,12,24,-9
56868,80,2,05.01.16,10421,12,11,2707,21,TUEF,05.01.16,86346,86456,86358,86456,12,2934,21,WINH,05.01.16,86394,86482,86406,86482,97581,4827,4,1,4,TRIE - BOEN,113576,2580,2710,6311,6438,6438,21,2580,47.398573,8.495001,13.0,True,21,2710,47.400142,8.494594,297.0,True,NaN,12,36,110
74586,308,2,05.01.16,11447,16,27,2934,20,WINH,05.01.16,86358,86291,86370,86291,28,2707,22,TUEF,05.01.16,86394,86333,86406,86333,62634,23175,2,1,2,BWEI - BALN ab j16,113388,2710,2580,6437,6314,6314,20,2710,47.399998,8.494698,115.0,True,22,2580,47.398037,8.494685,191.0,False,NaN,12,24,-67


# Map Visualization

In [23]:
import folium
# import geojson
import os

In [24]:
m = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")

# Use "OpenStreetMap" for more detail  -  tiles

# Tag
tooltip = 'Click for Signal'

temp = b

In [25]:
unique_GPSs = temp[['GPS_Latitude_x','GPS_Longitude_x']]
unique_GPSs.drop_duplicates(inplace=True)
tuples = [tuple(x) for x in unique_GPSs.values]
# tuples

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
# Visualize every stelle - agg punkts via mean to get stelles
# Fixing the gps first:

subdf = temp.groupby('halt_id_x').agg(np.mean)
# subdf.columns.values

In [27]:
unique_GPSs = subdf[['GPS_Latitude_x','GPS_Longitude_x']]
unique_GPSs.drop_duplicates(inplace=True)
tuples = [tuple(x) for x in unique_GPSs.values]

for i in tuples:
#     print(i[0].replace(',', '.'))
    # Create Markers 
    folium.Circle([i[0], i[1]], radius= 100).add_to(m)

# Make map
m.save('map.html')

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
temp['fahrweg_id'].value_counts()
# temp_2 = temp[temp['fahrweg_id'] ==18305]
temp_2 = temp.copy(deep=True)
len(set(temp_2['GPS_Latitude_x']))
# temp_2 = temp_2.groupby(['halt_id_x', 'seq_von']).agg(np.mean)
temp_2.sort_values(by = ['fahrt_id', 'seq_von'])

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y,intervals,target_stationary,target_travel_time,delay
102180,2,2,05.01.16,3016,1,1,3041,0,BTIE,05.01.16,56190,56215,56190,56215,2,2907,1,WILD,05.01.16,56250,56264,56262,56279,44245,18070,2,1,2,BTIE - FARB,112368,2788,2651,6643,6055,6055,0,2788,47.350565,8.560930,290.0,False,1,2651,47.353376,8.558699,330.0,True,63,0,60,25
18643,2,2,05.01.16,3016,1,2,2907,1,WILD,05.01.16,56250,56264,56262,56279,3,917,1,FROE,05.01.16,56304,56321,56322,56335,44245,18070,2,1,2,BTIE - FARB,112368,2651,2680,6055,6371,6371,1,2651,47.353376,8.558699,330.0,True,1,2680,47.355373,8.556809,327.0,False,63,12,42,14
16985,2,2,05.01.16,3016,1,3,917,1,FROE,05.01.16,56304,56321,56322,56335,4,1286,1,HOES,05.01.16,56376,56392,56394,56410,44245,18070,2,1,2,BTIE - FARB,112368,2680,2615,6371,5961,5961,1,2680,47.355373,8.556809,327.0,False,1,2615,47.358061,8.554178,331.0,False,63,18,54,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99666,80,1,05.01.16,10276,19,25,47,20,ALBR,05.01.16,85806,85742,85818,85742,26,1007,20,GOLD,05.01.16,85872,85797,85884,85797,97620,18627,3,1,3,BOEN - TRIE,111701,2810,2718,6697,6470,6470,20,2810,47.374463,8.484526,214.0,True,20,2718,47.371942,8.487773,129.0,True,NaN,12,54,-64
159827,80,1,05.01.16,10276,19,26,1007,20,GOLD,05.01.16,85872,85797,85884,85797,27,1320,20,IDEY,05.01.16,85926,85833,85938,85833,97620,18627,3,1,3,BOEN - TRIE,111701,2718,2851,6470,6799,6799,20,2718,47.371942,8.487773,129.0,True,20,2851,47.369519,8.491049,125.0,True,NaN,12,42,-75
97973,80,1,05.01.16,10276,19,27,1320,20,IDEY,05.01.16,85926,85833,85938,85833,28,2680,21,TRIE,05.01.16,85992,85884,85992,86541,97620,18627,3,1,3,BOEN - TRIE,111701,2851,1270,6799,2743,2743,20,2851,47.369519,8.491049,125.0,True,21,1270,47.368020,8.495680,73.0,True,NaN,12,54,-93


In [29]:
# Continue With Map Visualization

In [30]:
# Creating a pair of gps locations and calculating the signal at that time instance:
# temp['Road_Segment_from'] = list(zip(temp.GPS_Latitude_x, temp.GPS_Longitude_x))
# temp['Road_Segment_to'] = list(zip(temp.GPS_Latitude_y, temp.GPS_Longitude_y))
# temp

# aggregating the gps locations of the same punkts
df_new_x = temp.groupby(['halt_id_x'], as_index=False)['GPS_Latitude_x', 'GPS_Longitude_x'].mean()
df_new_y = temp.groupby(['halt_id_y'], as_index=False)['GPS_Latitude_y', 'GPS_Longitude_y'].mean()
# Merging the two frames signifying a road segment
final = temp.merge(df_new_x, on=['halt_id_x'])
final = final.merge(df_new_y, on=['halt_id_y'])


In [31]:
final.rename(columns={'GPS_Latitude_x_y':'GPS_LAN_STELLE_FROM', 'GPS_Longitude_x_y':'GPS_LON_STELLE_FROM',
                      'GPS_Latitude_y_y':'GPS_LAN_STELLE_TO', 'GPS_Longitude_y_y':'GPS_LON_STELLE_TO'}, inplace=True)
final

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,intervals,target_stationary,target_travel_time,delay,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO
0,17,2,05.01.16,2083,2,4,467,1,BERO,05.01.16,18006,17991,18018,17996,5,2447,1,SPOH,05.01.16,18078,18055,18090,18059,48512,6736,6,1,6,EWYS - WERD für Ausfahrt,114055,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,0,12,60,-15,47.393838,8.508641,47.394216,8.504226
1,17,2,05.01.16,2071,4,4,467,1,BERO,05.01.16,18906,18950,18918,18952,5,2447,1,SPOH,05.01.16,18978,19010,18990,19012,48586,6736,6,1,6,EWYS - WERD für Ausfahrt,113676,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,1,12,60,44,47.393838,8.508641,47.394216,8.504226
2,17,2,05.01.16,2085,6,14,467,1,BERO,05.01.16,19674,19644,19686,19652,15,2447,1,SPOH,05.01.16,19746,19715,19758,19724,48656,18449,8,1,8,BEZI - WERD für Ausfahrt,111525,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,2,12,60,-30,47.393838,8.508641,47.394216,8.504226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209127,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,65070,65325,65088,65325,4,2239,50,SLIM,05.01.16,65136,65369,65154,65369,59201,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,78,18,48,255,47.326188,8.539475,47.324670,8.542949
209128,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,68670,68920,68688,68927,4,2239,50,SLIM,05.01.16,68736,68981,68754,68981,59205,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,84,18,48,250,47.326188,8.539475,47.324670,8.542949
209129,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,72270,72358,72288,72358,4,2239,50,SLIM,05.01.16,72336,72412,72354,72412,59209,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,90,18,48,88,47.326188,8.539475,47.324670,8.542949


In [32]:
final['Road_Segment_From'] = list(zip(final.GPS_LAN_STELLE_FROM, final.GPS_LON_STELLE_FROM))
final['Road_Segment_To'] = list(zip(final.GPS_LAN_STELLE_TO, final.GPS_LON_STELLE_TO))
final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
final['Road_Segment_Signal'] = final[actual_arrival_to] - final[actual_departure_from]
final

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,intervals,target_stationary,target_travel_time,delay,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Road_Segment_Signal
0,17,2,05.01.16,2083,2,4,467,1,BERO,05.01.16,18006,17991,18018,17996,5,2447,1,SPOH,05.01.16,18078,18055,18090,18059,48512,6736,6,1,6,EWYS - WERD für Ausfahrt,114055,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,0,12,60,-15,47.393838,8.508641,47.394216,8.504226,"(47.39383817857139, 8.508641207142878)","(47.39421607117432, 8.50422574733094)","((47.39383817857139, 8.508641207142878), (47.3...",59
1,17,2,05.01.16,2071,4,4,467,1,BERO,05.01.16,18906,18950,18918,18952,5,2447,1,SPOH,05.01.16,18978,19010,18990,19012,48586,6736,6,1,6,EWYS - WERD für Ausfahrt,113676,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,1,12,60,44,47.393838,8.508641,47.394216,8.504226,"(47.39383817857139, 8.508641207142878)","(47.39421607117432, 8.50422574733094)","((47.39383817857139, 8.508641207142878), (47.3...",58
2,17,2,05.01.16,2085,6,14,467,1,BERO,05.01.16,19674,19644,19686,19652,15,2447,1,SPOH,05.01.16,19746,19715,19758,19724,48656,18449,8,1,8,BEZI - WERD für Ausfahrt,111525,2645,2572,6042,6283,6283,1,2645,47.393863,8.508392,271.0,False,1,2572,47.394236,8.504015,276.0,False,2,12,60,-30,47.393838,8.508641,47.394216,8.504226,"(47.39383817857139, 8.508641207142878)","(47.39421607117432, 8.50422574733094)","((47.39383817857139, 8.508641207142878), (47.3...",63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209127,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,65070,65325,65088,65325,4,2239,50,SLIM,05.01.16,65136,65369,65154,65369,59201,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,78,18,48,255,47.326188,8.539475,47.324670,8.542949,"(47.326188, 8.539474999999998)","(47.32467, 8.542949)","((47.326188, 8.539474999999998), (47.32467, 8....",44
209128,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,68670,68920,68688,68927,4,2239,50,SLIM,05.01.16,68736,68981,68754,68981,59205,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,84,18,48,250,47.326188,8.539475,47.324670,8.542949,"(47.326188, 8.539474999999998)","(47.32467, 8.542949)","((47.326188, 8.539474999999998), (47.32467, 8....",54
209129,163,1,05.01.16,9018,1,3,6782,50,KBRE,05.01.16,72270,72358,72288,72358,4,2239,50,SLIM,05.01.16,72336,72412,72354,72412,59209,12792,1,1,1,KOBE - BKIL,114476,2500,1538,5734,3431,3431,50,2500,47.326188,8.539475,91.0,False,50,1538,47.324670,8.542949,167.0,True,90,18,48,88,47.326188,8.539475,47.324670,8.542949,"(47.326188, 8.539474999999998)","(47.32467, 8.542949)","((47.326188, 8.539474999999998), (47.32467, 8....",54


In [33]:
# list of all of different stelles
list_of_stelles = list(set(final.halt_id_x))
list_of_node_signals = []

for i in list_of_stelles:
    temp_df = final[final.halt_id_x == i]
    signal = temp_df.Road_Segment_Signal
    Node_Signal = {i:np.array(signal)}
    list_of_node_signals.append(Node_Signal)
# list_of_node_signals

In [34]:
m = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")

# Use "OpenStreetMap" for more detail  -  tiles

# Tag
tooltip = 'Click for Signal'


In [35]:
unique_GPSs = temp[['GPS_Latitude_x','GPS_Longitude_x']]
unique_GPSs.drop_duplicates(inplace=True)
tuples = [tuple(x) for x in unique_GPSs.values]
# tuples

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
# All of the Punkts

In [37]:
# Just the punkt:
for i in tuples:
    # Create Markers 
    folium.Marker([i[0], i[1]], tooltip = tooltip).add_to(m)

# Make map
m.save('map_All_punkts.html')

In [38]:
# Visualize every stelle - agg punkts via mean to get stelles
# Fixing the gps first:

subdf = temp.groupby('halt_id_x').agg(np.mean)
# subdf.columns.values

In [39]:
# All hubs

In [40]:
m = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")

unique_GPSs = subdf[['GPS_Latitude_x','GPS_Longitude_x']]
unique_GPSs.drop_duplicates(inplace=True)
tuples = [tuple(x) for x in unique_GPSs.values]

for i in tuples:
#     print(i[0].replace(',', '.'))
    # Create Markers 
    folium.Circle([i[0], i[1]], radius= 100).add_to(m)

# Make map
m.save('map_All_Stelle.html')
# m

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [41]:
# Visualize just one route - the one with the most number of points

In [42]:
# Find the most common route:
temp['fahrweg_id'].value_counts()[:5]

18305    4607
18306    4479
21424    4464
21423    4280
18348    3941
Name: fahrweg_id, dtype: int64

In [43]:
# Just one route - most used one

In [44]:
temp_2 = temp[temp['fahrweg_id'] ==21424]
len(set(temp_2['GPS_Latitude_x']))
temp_2 = temp_2.groupby(['halt_id_x', 'seq_von']).agg(np.mean)
temp_2.sort_values(by='seq_von', inplace=True)
temp_2

,,linie,richtung,fahrzeug,kurs,halt_diva_von,halt_punkt_diva_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,GPS_Latitude_x,GPS_Longitude_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y,GPS_Longitude_y,GPS_Bearing_y,halt_punkt_ist_aktiv_y,target_stationary,target_travel_time,delay
halt_id_x,seq_von,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1270,1,33.0,2.0,10176.511111,6.725926,2680.0,10.0,50514.666667,50529.822222,50514.666667,50529.822222,2.0,1320.0,11.0,50574.666667,50606.229630,50587.511111,50618.362963,49951.044444,21424.0,2.0,1.0,2.0,113154.577778,1270.0,2851.0,2744.0,6802.0,6802.0,10.0,47.368074,8.495694,73.0,False,11.0,2851.0,47.369683,8.491913,351.0,True,0.000000,60.000000,15.155556
2851,2,33.0,2.0,10176.604478,6.708955,1320.0,11.0,50722.164179,50744.425373,50735.014925,50756.649254,3.0,3155.0,11.0,50789.014925,50811.589552,50801.865672,50824.231343,49946.746269,21424.0,2.0,1.0,2.0,113163.111940,2851.0,2840.0,6802.0,6770.0,6770.0,11.0,47.369683,8.491913,351.0,True,11.0,2840.0,47.372738,8.494251,15.0,True,12.850746,54.000000,22.261194
2840,3,33.0,2.0,10176.604478,6.708955,3155.0,11.0,50789.014925,50811.589552,50801.865672,50824.231343,4.0,2147.0,11.0,50831.865672,50853.410448,50845.611940,50867.052239,49946.746269,21424.0,2.0,1.0,2.0,113163.111940,2840.0,2829.0,6770.0,9364.0,9364.0,11.0,47.372738,8.494251,15.0,True,11.0,2829.0,47.374175,8.495431,32.0,True,12.850746,30.000000,22.574627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,31,33.0,2.0,10181.370370,6.688889,1286.0,11.0,52393.288889,52450.385185,52407.066667,52462.881481,32.0,917.0,0.0,52477.600000,52535.392593,52489.600000,52543.740741,49944.325926,21424.0,2.0,1.0,2.0,113163.874074,2615.0,2680.0,5963.0,6373.0,6373.0,11.0,47.357884,8.554836,251.0,True,0.0,2680.0,47.355065,8.557001,147.0,False,13.777778,70.533333,57.096296
2680,32,33.0,2.0,10181.370370,6.688889,917.0,0.0,52477.600000,52535.392593,52489.600000,52543.740741,33.0,2907.0,0.0,52525.600000,52580.933333,52537.600000,52587.859259,49944.325926,21424.0,2.0,1.0,2.0,113163.874074,2680.0,2651.0,6373.0,6056.0,6056.0,0.0,47.355065,8.557001,147.0,False,0.0,2651.0,47.353104,8.558893,146.0,False,12.000000,36.000000,57.792593
2651,33,33.0,2.0,10181.370370,6.688889,2907.0,0.0,52525.600000,52580.933333,52537.600000,52587.859259,34.0,3041.0,10.0,52644.133333,52697.792593,52644.133333,52988.140741,49944.325926,21424.0,2.0,1.0,2.0,113163.874074,2651.0,2788.0,6056.0,6639.0,6639.0,0.0,47.353104,8.558893,146.0,False,10.0,2788.0,47.350216,8.560778,290.0,False,12.000000,106.533333,55.333333


In [45]:
# Routes
m = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")
# Use weight as how many datapoints I have or how often it is used

# Get list of unique routes:

unique_routes = list(set(temp_2['fahrweg_id']))

# Plot them:
for i in unique_routes:

    temp_3 = temp_2[temp_2['fahrweg_id'] == i]
    if len(temp_2) == 1 or 0:
        continue
    list_of_corrd = list(zip(temp_3['GPS_Latitude_x'], temp_3['GPS_Longitude_x']))
    color = [i for i in range(len(list_of_corrd) - 1)]
    if not color:
        continue
    folium.ColorLine(list_of_corrd, colors=color, colormap=('#00008B', '#00BFFF'), weight=4).add_to(m)
    # Create Markers 
    folium.Marker([47.418375, 8.561418], popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color='purple')).add_to(m)

# Make map
m.save('map_Just_One_Route.html')

In [46]:
# Question: Finding whether there are routes where a pair of punkts are successive stops

In [47]:
# Get top route id based on halt id from (x)
final.halt_id_x.value_counts()[:5]

1565    2104
1938    2066
1528    1885
1311    1571
1309    1549
Name: halt_id_x, dtype: int64

In [48]:
list_of_routes = list(set(final.fahrweg_id))

for i in list_of_routes:
    new_df = final[(final.fahrweg_id == i) & (1565 in final.halt_id_x)]
#     print(new_df)


In [49]:
m2 = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")

# Use "OpenStreetMap" for more detail  -  tiles

# Tag
tooltip = 'Click for Signal'

In [50]:
# Most used route
final.fahrweg_id.value_counts()[:5]

18305    4607
18306    4479
21424    4464
21423    4280
18348    3941
Name: fahrweg_id, dtype: int64

In [51]:
temp_3 = final[(final.fahrweg_id == 18305) & (1565 in final.halt_id_x)]
temp_3 = temp_3.groupby(by=['seq_nach', 'halt_id_x']).agg(np.mean)
temp_3.sort_values(by='seq_nach')
list_of_corrd = list(zip(temp_3.GPS_Latitude_x_x, temp_3.GPS_Longitude_x_x))
print(np.array(list_of_corrd)[:5])

[[47.416685  8.568764]
 [47.417118  8.568561]
 [47.418375  8.561418]
 [47.420254  8.556911]
 [47.41746   8.553752]]


In [52]:
m = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")

list_of_routes = list(set(final.halt_id_x))

# # Plot them:


list_of_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'black', 'lightgray']

index_of_color = 0
for i in list_of_routes:
    temp_3 = final[final.halt_id_x == i]
#     temp_3 = temp_3[temp_3['halt_id_x'].isin([1565])]
#     temp_3 = new_df[new_df['fahrweg_id'] == i]
#     print(temp_3, '\n')
#     print(temp_3.GPS_LAN_STELLE_FROM)
#     temp_3.sort_values(by='seq_von')
#     print(tem)

    if temp_3.empty:
        continue
#     print(temp_3)
#     print(temp_3)

#     temp_3 = temp_3.groupby(by=['seq_nach']).agg(np.mean)
#     print(temp_3.head())
    halt_coord = [set(temp_3.GPS_LAN_STELLE_FROM), set(temp_3.GPS_LAN_STELLE_FROM)]
    list_of_corrd = list(zip(set(temp_3.GPS_Latitude_x_x), set(temp_3.GPS_Longitude_x_x)))
#     color = [i for i in range(len(list_of_corrd) - 1)]
#     if not color:
#         continue
# #     print(list_of_corrd)
#     folium.ColorLine(list_of_corrd, colors=color, colormap=('#00008B', '#00BFFF'), weight=4).add_to(m2)
    
    
    for j in list_of_corrd:
        folium.Marker(j, popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color=list_of_colors[i % len(list_of_colors)])).add_to(m2)
#     index_of_color += 1
print(list_of_corrd)
    # Create Markers 
#     folium.Marker(list_of_corrd, popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color='purple')).add_to(m2)
folium.Marker([47.367029, 8.545065], popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color='purple')).add_to(m2)


# print(list_of_corrd)



# Make map
m2.save('map_haltestelles_and_punkts.html')

[(47.383144, 8.54349), (47.382748, 8.543379)]


In [53]:
# For a given route:
temp_d = list(set(final[final.fahrweg_id == 18305].Road_Segment_Node))
# print(temp_d)
list_of_punkts_connected = []
for i in temp_d:
    df = final[final.Road_Segment_Node == i]
    a = df[df.GPS_Latitude_x_x != df.GPS_Latitude_y_x]
    list_of_punkts_connected.append(len(a.index))
# list_of_punkts_connected

In [54]:
m3 = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")
fahrweg_id_to_look_for = final[final.halt_id_x==1565].fahrweg_id.value_counts().index
for i in fahrweg_id_to_look_for:

    a = final[final.fahrweg_id == i]
    if 1565 in set(a.halt_id_x):
        a = a.groupby(by='seq_von').agg(np.mean)

        if a.empty or len(a) < 2:
            continue

        list_of_corrd = list(zip(a.GPS_Latitude_x_x, a.GPS_Longitude_x_x))

        list_of_corrd_halt = list(zip(a.GPS_LAN_STELLE_FROM, a.GPS_LON_STELLE_FROM))

        color = [i for i in range(len(list_of_corrd) - 1)]
        if not color:
            continue
        folium.ColorLine(list_of_corrd, colors=color, colormap=('#00008B', '#00BFFF'), weight=4).add_to(m3)
        for j in list_of_corrd_halt:
            folium.Circle(j, popup='<strong>Location One</strong>', tooltip=tooltip, radius=100).add_to(m3)


        for j in list_of_corrd:
            folium.Marker(j, popup='<strong>Location One</strong>', tooltip=tooltip, icon=folium.Icon(icon='cloud', color=list_of_colors[i % len(list_of_colors)])).add_to(m3)
    index_of_color += 1

# list_of_punkts_in_stelle = list(zip(list(set(final[final.halt_id_x == 1565].GPS_Latitude_x_x)), list(set(final[final.halt_id_x == 1565].GPS_Longitude_x_x))))
# for i in list_of_punkts_in_stelle:
#     folium.Marker([i[0], i[1]], popup='<strong>Location One</strong>', tooltip=tooltip).add_to(m2)


# print(list_of_corrd)
# Create Markers
#     folium.Marker(list_of_corrd, popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color='purple')).add_to(m2)
folium.Marker([47.367029, 8.545065], popup='<strong>Location One</strong>', tooltip=tooltip).add_to(m3)

# print(list_of_corrd)


# Make map
m3.save('map_setteles_for_oneRoute_Common.html')